# Limpeza de dados

In [1]:
%load_ext autoreload
%autoreload 2


import nltk
import pandas as pd
import string
from nltk.corpus import stopwords 
from nltk.tokenize import TweetTokenizer



In [2]:
pd.set_option('max_colwidth', 300)

In [32]:
!python -m spacy download pt

[autoreload of pkg_resources failed: Traceback (most recent call last):
  File "/home/lsbpaiva/.pyenv/versions/3.8.12/envs/twitterelectionbr/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "/home/lsbpaiva/.pyenv/versions/3.8.12/envs/twitterelectionbr/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 455, in superreload
    module = reload(module)
  File "/home/lsbpaiva/.pyenv/versions/3.8.12/lib/python3.8/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 604, in _exec
  File "<frozen importlib._bootstrap_external>", line 843, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/home/lsbpaiva/.pyenv/versions/3.8.12/envs/twitterelectionbr/lib/python3.8/site-packages/pkg_resources/__init__.py", line 3095, in <module>
    class RequirementParseError(packaging.requir

⚠ As of spaCy v3.0, shortcuts like 'pt' are deprecated. Please use the
full pipeline package name 'pt_core_news_sm' instead.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 2.4 MB/s eta 0:00:0000:0100:01m
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [3]:
df = pd.read_csv('../raw_data/dilma.csv')



/tmp/ipykernel_3798/1274174624.py:1: DtypeWarning: Columns (1,2,5,6,7,8,9,10,11,12,13,14,15,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../raw_data/dilma.csv')


In [1]:
data = pd.read_csv('../raw_data/biroliro_total_tweets.csv')

NameError: name 'pd' is not defined

In [5]:
data.head().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   url              5 non-null      object
 1   date             5 non-null      object
 2   content          5 non-null      object
 3   id               5 non-null      int64 
 4   reply_count      5 non-null      int64 
 5   retweet_count    5 non-null      int64 
 6   like_count       5 non-null      int64 
 7   quote_count      5 non-null      int64 
 8   lang             5 non-null      object
 9   username         5 non-null      object
 10  displayname      5 non-null      object
 11  description      3 non-null      object
 12  verified         5 non-null      bool  
 13  created          5 non-null      object
 14  followers_count  5 non-null      int64 
 15  friends_count    5 non-null      int64 
 16  location         5 non-null      object
 17  protected        5 non-null      bool  

In [6]:
#filtro = df[(df.lang == 'pt') & (df.url.str.startswith('https'))]
#df = filtro.reset_index(drop = True)

## Transformadores Personalizados

### Limpeza colunas de texto

In [3]:
from sklearn.pipeline import make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import  make_column_transformer
import re 

class CleaningEncoder(BaseEstimator, TransformerMixin):
    '''
    Receives raw text data from the tweets and returns clean, ready to process data:
    turns all into lower case;
    removes punctuation;
    removes stopwords;
    removes numbers;
    removes users' handles
    
    '''
    def __init__(self):
        pass
                        
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        #assert isinstance(X, pd.DataFrame)
        X_ = X.copy()      
        
        #removin NaN values
        X_ = X_.fillna('nenhuma descrição')
        
        #removing links
        X_ = X_.apply(lambda x: re.sub(r"http\S+", "", x))
        X_ = X_.apply(lambda x: re.sub(r"www.\S+", "", x))
                
        #removing punctuation from each tweet
        new_punc = list(string.punctuation)
        del new_punc[2]
        del new_punc[-11]
        for punctuation in new_punc:
            X_ = X_.str.replace(punctuation, '')
            
        #removing numbers
        X_ = X_.str.replace('\d+', '')
        
        #tokenizing - removes handles, applies lowercase, keeps #, shortens letter repetitions to three
        #ex: kkkkk, kkkkkk, kkkkkkkkk = kkk
        tkn = TweetTokenizer(preserve_case=False, reduce_len=True, strip_handles=True)
        X_ = X_.apply(lambda x: tkn.tokenize(x))
        
        #removing stopwords
        stop_words = stopwords.words('portuguese')
        stop_words.remove('não')
        addicional = [
            'ta', 'q', 'nao', 'tah', 'tao', 'eh', 'vc', 'voce',
            'pq', 'quedê', 'mane', 'mto', 'mt', 'bj', 'bjs',
            'b', 'sao', 'axo', 'mano', 'ae', 'neh', 'aí',
            'kkk', 'porque', 'né', 'no']
        stop_words.extend(addicional)
        
        X_ = X_.apply(lambda x: [word for word in x if word not in (stop_words)])
        
        return X_

In [4]:
#.str.replace('\d+', '')
X = data.content
#X.str.lower()

#dataframe['column_name']=dataframe['column_name'].apply(str)



NameError: name 'data' is not defined

In [10]:
clean = CleaningEncoder()
X_clean = clean.fit_transform(X)

X_clean


/tmp/ipykernel_3798/2400588789.py:39: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  X_ = X_.str.replace(punctuation, '')
/tmp/ipykernel_3798/2400588789.py:42: FutureWarning: The default value of regex will change from True to False in a future version.
  X_ = X_.str.replace('\d+', '')


0                                                                                [vai, acontecer, biroliro]
1                                  [passei, min, metrô, cercada, macho, babaca, falando, biroliro, inferno]
2       [tô, adotando, postura, ciro, pessoa, vem, tentar, convencer, biroliro, não, facista, mando, “, ...
3                         [facil, pro, eleitor, indeciso, escolher, ciro, haddad, biroliro, segundo, turno]
4                                                   [rompendo, laços, familiares, causa, biroliro, #sextou]
                                                       ...                                                 
3530                                                                 [ousam, dizer, homem, igual, biroliro]
3531                                                           [biroliro, bonoro, bozonaro, bololo, boniro]
3532                                             [pronto, achei, melhor, apelido, pro, salnorabo, biroliro]
3533                        

### Transformação da data

In [29]:
class TimeEncoder(BaseEstimator, TransformerMixin):
    '''
    Transforms the date column from string to datetime object 
    '''
    
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X_ = X.copy()
        X_ = X_.apply(lambda x: pd.to_datetime(x))
        X_ = pd.DataFrame(X_)
        X_['data'] = X_['date'].dt.date
        X_['hora'] = X_['date'].dt.time
        X_.drop(columns = 'date', inplace=True)
        return X

In [42]:
X_test = data['date']
timer = TimeEncoder()
result = timer.fit_transform(X_test)

# Função latitude longitude


In [61]:
data = pd.read_csv('../raw_data/vader_nlp_datasets/#haddad_vader.csv')

In [159]:
teste = data.head(20)

In [106]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent = "my project app")

In [160]:
data.location.fillna('Brasil', inplace=True)

/tmp/ipykernel_754/1171173744.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  teste.location.fillna('Brasil', inplace=True)


In [166]:
dado_teste.location.fillna('Brasil', inplace=True)

In [ ]:
data.location = data.location.replace('','Brasil' )

In [124]:
data.location.value_counts()

Brasil                    5580
São Paulo, Brasil          737
Rio de Janeiro, Brasil     509
São Paulo                  506
Rio de Janeiro             355
                          ... 
Porto Alegre, Paris          1
Por Aí                       1
BH, Brazil                   1
Ipupiara, Brasil             1
Nova Iguaçu - RJ             1
Name: location, Length: 3015, dtype: int64

In [174]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent = "my project app")

def find_location(location):
    dados['location'].fillna('Brasil', inplace=True)
    
    try:
        local  = geolocator.geocode(location).point
        latitude = local[0]
        longitude = local[1]
    except:
        latitude = -10.3333333
        longitude = -53.2
    return latitude, longitude
        


In [171]:
def location_ready(dados):
    results = dados.location.apply(lambda x: find_location(x))
    dados[['latitude', 'longitude']] = pd.DataFrame(results.to_list())
    